In [10]:
import requests

In [11]:
medalists = 'https://en.wikipedia.org/w/api.php?action=query&titles=List%20of%20multiple%20Olympic%20medalists&prop=revisions&rvprop=content&format=json&formatversion=2'
medalists_page = requests.get(medalists)

In [12]:
j = medalists_page.json()

In [13]:
j['query'].keys()

dict_keys(['pages'])

In [14]:
j['query']['pages'][0].keys()

dict_keys(['pageid', 'ns', 'title', 'revisions'])

In [15]:
j['query']['pages'][0]['revisions'][0].keys()

dict_keys(['contentformat', 'contentmodel', 'content'])

In [16]:
j['query']['pages'][0]['revisions'][0]['content'][2000:3000]

'\n! style="width:3.5em;"|Total\n|-\n| 1\n| align=left | {{sortname|Michael|Phelps}} || align=left | {{flagIOC|USA}} || | [[Swimming at the Summer Olympics|Swimming]] || 2004–2016 || Summer || M || 23 || 3 || 2 || 28\n|-\n| 2\n| align=left | {{sortname|Larisa|Latynina}} || align=left | {{flagIOC|URS}} || | [[Gymnastics at the Summer Olympics|Gymnastics]] || 1956–1964 || Summer || F || 9 || 5 || 4 || 18\n|-\n| 3\n| align="left" | {{sortname|Marit|Bjørgen||Bjorgen, Marit}} || align="left" | {{flagIOC|NOR}} || | [[Cross-country skiing at the Winter Olympics|Cross-country skiing]] || 2002–2018 || Winter || F || 8 || 4 || 3 || rowspan="2" | 15\n|-\n| 4\n| align=left | {{sortname|Nikolai|Andrianov}} || align=left | {{flagIOC|URS}} || | [[Gymnastics at the Summer Olympics|Gymnastics]] || 1972–1980 || Summer || M || 7 || 5 || 3\n|-\n| 5 \n| align=left | {{sortname|Ole Einar|Bjørndalen||Bjorndalen, Ole Einar}} || align=left | {{flagIOC|NOR}} || | [[Biathlon at the Winter Olympics|Biathlon]] || 

In [17]:
import re
list1 = re.findall('sortname.+?}}',j['query']['pages'][0]['revisions'][0]['content'])

In [32]:
athletes = []
for x in list1:
    athletes.append(x.split('|')[1:3])

In [33]:
url1 = "https://en.wikipedia.org/w/api.php?action=query&titles="
url3 = "&prop=revisions&rvprop=content&format=json&formatversion=2"

In [236]:
urls = []
names = []
for name in athletes:
    urls.append(url1+name[0]+"%20"+name[1][:-2]+url3)
    names.append((name[0], name[1][:-2]))

urls[0], names[0]

('https://en.wikipedia.org/w/api.php?action=query&titles=Michael%20Phelps&prop=revisions&rvprop=content&format=json&formatversion=2',
 ('Michael', 'Phelps'))

In [37]:
first = requests.get(urls[0])

In [233]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [237]:
df = pd.DataFrame(urls, columns=['urls'])
df['names'] = names
df['responses'] = None
df.shape

(173, 3)

In [238]:
def try_request(x):
    try:
        return requests.get(x)
    except:
        return None

In [239]:
df['responses'] = df.urls.progress_apply(try_request)

100%|████████████████████████████████████████████████████████████████████████████████| 173/173 [00:34<00:00,  5.62it/s]


In [240]:
df.responses.apply(lambda x: x.ok==True).value_counts()

True    173
Name: responses, dtype: int64

In [241]:
# df.responses.apply(lambda x: x.json())

In [242]:
df['pages'] = df.responses.apply(lambda x: x.json()['query']['pages'][0])

In [243]:
df['found'] = df.pages.apply(lambda x: 'missing' not in x.keys())

In [244]:
def find_height(x):
    try:
        return re.search('height\s*=\s*(\d*.*?\d\s..)', x.lower()).groups()[0]
    except:
        return None
    
# df[df['found']].pages.apply(lambda x: x['revisions'][0]['content']).apply(find_height)

In [245]:
def find_height(x):
    x = x.lower()
    
    meter = re.search('m=(\d\.\d\d)', x)
    if meter != None:
        return (meter.groups()[0],'m')
    
    
    meter = re.search('cm=(\d*)', x)
    if meter != None:
        return (meter.groups()[0],'cm')
    
    ft_in = re.search('height.*(\d+).*ft.*(\d+).*in', x)
    if ft_in != None:
        return list(zip(ft_in.groups(),('ft','in')))
    
    meters = re.search('height.*(\d+?\.\d+).*m', x)
    if meters != None:
        return (meters.groups()[0],'m')
    
    cm = re.search('height.*?(\d+).?cm', x)
    if cm != None:
        return (cm.groups()[0], 'cm')
    
    height = re.search('height.*\d+', x)
    if height != None:
        return height.group()

df['height'] = None
df2 = df[df['found']].copy()
df2['height'] = df2.pages.apply(lambda x: x['revisions'][0]['content']).apply(find_height)
df.update(df2)
df2=None
df.tail(5)

,urls,names,responses,pages,found,height
168,https://en.wikipedia.org/w/api.php?action=quer...,"(Kirsty, Coventry)",<Response [200]>,"{'pageid': 915950, 'ns': 0, 'title': 'Kirsty C...",True,"(1.76, m)"
169,https://en.wikipedia.org/w/api.php?action=quer...,"(Stefania, Belmondo)",<Response [200]>,"{'pageid': 3094787, 'ns': 0, 'title': 'Stefani...",True,"(158, cm)"
170,https://en.wikipedia.org/w/api.php?action=quer...,"(Ryan, Lochte)",<Response [200]>,"{'pageid': 4057953, 'ns': 0, 'title': 'Ryan Lo...",True,"[(6, ft), (2, in)]"
171,https://en.wikipedia.org/w/api.php?action=quer...,"(Andrea, Ehrig-Mitscherlich)",<Response [200]>,"{'pageid': 7759395, 'ns': 0, 'title': 'Andrea ...",True,"(1.65, m)"
172,https://en.wikipedia.org/w/api.php?action=quer...,"(Merlene, Ottey)",<Response [200]>,"{'pageid': 558060, 'ns': 0, 'title': 'Merlene ...",True,"(175, cm)"


In [246]:
def find_weight(x):
    x = x.lower()
    
    lbs = re.search('(\d+).?lb', x)
    if lbs != None:
        return (lbs.groups()[0], 'lbs')
    
    kgs = re.search('(\d+).?kg', x)
    if kgs != None:
        return (kgs.groups()[0], 'kg')
    
df['weight'] = None
df2 = df[df['found']].copy()
df2['weight'] = df2.pages.apply(lambda x: x['revisions'][0]['content']).apply(find_weight)
df.update(df2)
df2=None
df.tail(5)

,urls,names,responses,pages,found,height,weight
168,https://en.wikipedia.org/w/api.php?action=quer...,"(Kirsty, Coventry)",<Response [200]>,"{'pageid': 915950, 'ns': 0, 'title': 'Kirsty C...",True,"(1.76, m)","(64, kg)"
169,https://en.wikipedia.org/w/api.php?action=quer...,"(Stefania, Belmondo)",<Response [200]>,"{'pageid': 3094787, 'ns': 0, 'title': 'Stefani...",True,"(158, cm)",None
170,https://en.wikipedia.org/w/api.php?action=quer...,"(Ryan, Lochte)",<Response [200]>,"{'pageid': 4057953, 'ns': 0, 'title': 'Ryan Lo...",True,"[(6, ft), (2, in)]","(186, lbs)"
171,https://en.wikipedia.org/w/api.php?action=quer...,"(Andrea, Ehrig-Mitscherlich)",<Response [200]>,"{'pageid': 7759395, 'ns': 0, 'title': 'Andrea ...",True,"(1.65, m)","(58, kg)"
172,https://en.wikipedia.org/w/api.php?action=quer...,"(Merlene, Ottey)",<Response [200]>,"{'pageid': 558060, 'ns': 0, 'title': 'Merlene ...",True,"(175, cm)","(62, kg)"


In [247]:
t = df[df['found']].pages.apply(lambda x: x['revisions'][0]['content'])[0]

In [248]:
re.search('(\d+).*lb', t)

<_sre.SRE_Match object; span=(1436, 1442), match='194 lb'>

In [249]:
df

,urls,names,responses,pages,found,height,weight
0,https://en.wikipedia.org/w/api.php?action=quer...,"(Michael, Phelps)",<Response [200]>,"{'pageid': 19084502, 'ns': 0, 'title': 'Michae...",True,"[(6, ft), (4, in)]","(194, lbs)"
1,https://en.wikipedia.org/w/api.php?action=quer...,"(Larisa, Latynina)",<Response [200]>,"{'pageid': 73697, 'ns': 0, 'title': 'Larisa La...",True,"(161, cm)","(52, kg)"
2,https://en.wikipedia.org/w/api.php?action=quer...,"(Marit, Bjørg)",<Response [200]>,"{'ns': 0, 'title': 'Marit Bjørg', 'missing': T...",False,None,None
3,https://en.wikipedia.org/w/api.php?action=quer...,"(Nikolai, Andrianov)",<Response [200]>,"{'pageid': 1529185, 'ns': 0, 'title': 'Nikolai...",True,"(166, cm)","(60, kg)"
4,https://en.wikipedia.org/w/api.php?action=quer...,"(Ole Einar, Bjørndal)",<Response [200]>,"{'ns': 0, 'title': 'Ole Einar Bjørndal', 'miss...",False,None,None
5,https://en.wikipedia.org/w/api.php?action=quer...,"(Boris, Shakhlin)",<Response [200]>,"{'pageid': 3021184, 'ns': 0, 'title': 'Boris S...",True,"(1.71, m)","(71, kg)"
6,https://en.wikipedia.org/w/api.php?action=quer...,"(Edoardo, Mangiarotti)",<Response [200]>,"{'pageid': 2285970, 'ns': 0, 'title': 'Edoardo...",True,None,None
7,https://en.wikipedia.org/w/api.php?action=quer...,"(Takashi, Ono)",<Response [200]>,"{'pageid': 4863915, 'ns': 0, 'title': 'Takashi...",True,"(1.60, m)","(58, kg)"
8,https://en.wikipedia.org/w/api.php?action=quer...,"(Paavo, Nurmi)",<Response [200]>,"{'pageid': 25071, 'ns': 0, 'title': 'Paavo Nur...",True,"[(8, ft), (9, in)]","(65, kg)"
9,https://en.wikipedia.org/w/api.php?action=quer...,"(Birgit, Fischer)",<Response [200]>,"{'pageid': 1532025, 'ns': 0, 'title': 'Birgit ...",True,"(172, cm)","(69, kg)"


In [250]:
df.to_pickle('wiki_scrape')